In [7]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split,cross_val_score


In [11]:
df=pd.read_csv('Downloads/all_season_details.csv')

C:\Users\saihi\AppData\Local\Temp\ipykernel_22000\2602616471.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('Downloads/all_season_details.csv')


In [13]:
df.head(10)

,comment_id,season,match_id,match_name,home_team,away_team,current_innings,innings_id,over,ball,...,wicket_id,wkt_batsman_name,wkt_bowler_name,wkt_batsman_runs,wkt_batsman_balls,wkt_text,isRetiredHurt,text,preText,postText
0,110,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,False,"nice and full, angling and perhaps swinging in...",<p><strong>7.30pm</strong> Gaikwad and Conway ...,NaN
1,120,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,False,"Conway shuffles across off, and shimmies out a...",NaN,NaN
2,130,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,False,"good length, angling in at off and then straig...",NaN,NaN
3,140,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,False,"shorter and slanting into middle and leg, Gaik...","<p>Rummy: ""Fast bowling options for CSK defini...",NaN
4,150,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,False,Shami hits a heavy length and brings this back...,"<p>LG: ""Watching out for Hangargekar. He was i...",NaN
5,160,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,6,...,NaN,NaN,NaN,NaN,NaN,NaN,False,"on a good length, threatens to angle in, so Co...",NaN,NaN
6,11010,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,False,"back of a length and angling into middle, Gaik...",<p>Pandya takes the new ball. Over the wicket....,"<p>Ankush: ""I think Tushar Deshpande will be i..."
7,11020,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,False,"on a good length and sliding in at off stump, ...",NaN,NaN
8,11030,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,2,3,...,NaN,NaN,NaN,NaN,NaN,NaN,False,<strong>jabbing punch in the gap at covers</st...,"<p>Rajkumar Lingav: ""India have been searching...",NaN
9,11040,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,2,4,...,NaN,NaN,NaN,NaN,NaN,NaN,False,good length and angling into a front-foot push...,NaN,NaN


In [15]:
df.columns

Index(['comment_id', 'season', 'match_id', 'match_name', 'home_team',
       'away_team', 'current_innings', 'innings_id', 'over', 'ball', 'runs',
       'shortText', 'isBoundary', 'isWide', 'isNoball', 'batsman1_id',
       'batsman1_name', 'batsman1_runs', 'batsman1_balls', 'bowler1_id',
       'bowler1_name', 'bowler1_overs', 'bowler1_maidens', 'bowler1_runs',
       'bowler1_wkts', 'batsman2_id', 'batsman2_name', 'batsman2_runs',
       'batsman2_balls', 'bowler2_id', 'bowler2_name', 'bowler2_overs',
       'bowler2_maidens', 'bowler2_runs', 'bowler2_wkts', 'wicket_id',
       'wkt_batsman_name', 'wkt_bowler_name', 'wkt_batsman_runs',
       'wkt_batsman_balls', 'wkt_text', 'isRetiredHurt', 'text', 'preText',
       'postText'],
      dtype='object')

In [17]:
df['season'].fillna(2023, inplace=True)

C:\Users\saihi\AppData\Local\Temp\ipykernel_22000\3534781089.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['season'].fillna(2023, inplace=True)


In [19]:
df.loc[(df['match_id'].isin([1359496,1359538])) & (df['season'].isnull()), 'season'] = 2023

In [21]:
# Select the required columns and create a copy
ball_by_ball_df = df[['season', 'match_id', 'home_team', 'away_team', 'current_innings',
                      'innings_id', 'over', 'ball', 'runs', 'wicket_id', 
                      'wkt_batsman_name', 'wkt_batsman_runs']].copy()

# Step 1: Calculate cumulative runs for each match and innings
ball_by_ball_df['cumulative_runs'] = ball_by_ball_df.groupby(['match_id', 'current_innings'])['runs'].cumsum()

# Step 2: Calculate run rate by dividing cumulative runs by the over (adding a small value to avoid division by zero)
ball_by_ball_df['run_rate'] = ball_by_ball_df['cumulative_runs'] / (ball_by_ball_df['over'] + 0.1)

# Step 3: Calculate wickets lost by counting instances where `wkt_batsman_runs` is non-negative
ball_by_ball_df['wickets_lost'] = ball_by_ball_df.groupby(['match_id', 'current_innings'])['wkt_batsman_runs']\
                                                 .transform(lambda x: (x >= 0).cumsum())

# Step 4: Fill any NaN values in the DataFrame with 0
ball_by_ball_df.fillna(0, inplace=True)

In [23]:
ball_by_ball_df.head(25)

,season,match_id,home_team,away_team,current_innings,innings_id,over,ball,runs,wicket_id,wkt_batsman_name,wkt_batsman_runs,cumulative_runs,run_rate,wickets_lost
0,2023.0,1359475,GT,CSK,CSK,1,1,1,0,0.0,0,0.0,0,0.000000,0
1,2023.0,1359475,GT,CSK,CSK,1,1,2,1,0.0,0,0.0,1,0.909091,0
2,2023.0,1359475,GT,CSK,CSK,1,1,3,0,0.0,0,0.0,1,0.909091,0
3,2023.0,1359475,GT,CSK,CSK,1,1,4,1,0.0,0,0.0,2,1.818182,0
4,2023.0,1359475,GT,CSK,CSK,1,1,5,0,0.0,0,0.0,2,1.818182,0
5,2023.0,1359475,GT,CSK,CSK,1,1,6,0,0.0,0,0.0,2,1.818182,0
6,2023.0,1359475,GT,CSK,CSK,1,2,1,4,0.0,0,0.0,6,2.857143,0
7,2023.0,1359475,GT,CSK,CSK,1,2,2,0,0.0,0,0.0,6,2.857143,0
8,2023.0,1359475,GT,CSK,CSK,1,2,3,4,0.0,0,0.0,10,4.761905,0
9,2023.0,1359475,GT,CSK,CSK,1,2,4,1,0.0,0,0.0,11,5.238095,0


In [27]:
X = ball_by_ball_df[['over', 'ball', 'run_rate','wickets_lost']]
Y = ball_by_ball_df['cumulative_runs']

In [29]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [31]:
x_train

,over,ball,run_rate,wickets_lost
199032,12,4,5.123967,3
237647,17,1,8.596491,2
10462,1,5,4.545455,1
39475,8,3,8.395062,0
137928,9,1,6.153846,2
...,...,...,...,...
119879,10,5,7.029703,3
103694,12,3,8.016529,1
131932,5,6,9.607843,2
146867,18,5,7.182320,5


In [33]:
y_train

199032     62
237647    147
10462       5
39475      68
137928     56
         ... 
119879     71
103694     97
131932     49
146867    130
121958     74
Name: cumulative_runs, Length: 194040, dtype: int64

In [35]:
x_test

,over,ball,run_rate,wickets_lost
122566,10,4,4.455446,3
147656,5,1,5.294118,0
27099,20,5,9.402985,6
42493,2,4,0.952381,2
154359,18,6,5.856354,5
...,...,...,...,...
19877,17,4,8.304094,3
217199,14,2,9.503546,3
188029,2,6,6.190476,0
220380,14,6,6.241135,2


In [37]:
y_test

122566     45
147656     27
27099     189
42493       2
154359    106
         ... 
19877     142
217199    134
188029     13
220380     88
8762       52
Name: cumulative_runs, Length: 48510, dtype: int64

In [39]:
rf_model=RandomForestRegressor(n_estimators=200, max_depth=None, random_state=42,min_samples_leaf=1,min_samples_split=2)

In [43]:
rf_model.fit(x_train,y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

In [45]:
y_rf_simple_pred=rf_model.predict(x_test)

In [46]:
rf_mse=mean_squared_error(y_test,y_rf_simple_pred)
rf_r2=r2_score(y_test,y_rf_simple_pred)

In [49]:
rf_mse

0.06906843537414968

In [51]:
rf_r2

0.9999723813281342

In [53]:
new_data = pd.DataFrame({
    'over': [10],  
    'ball': [2],      
    'run_rate': [10],        
    'wickets_lost': [9]          
})

In [55]:
predicted_runs=rf_model.predict(new_data)
print(f"Predicted Cumulative Runs: {predicted_runs[0]}")

Predicted Cumulative Runs: 101.0


In [63]:
from sklearn.linear_model import LinearRegression


lr_model = LinearRegression()
lr_model.fit(x_train, y_train)

y_pred_lr = lr_model.predict(x_test)

mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)

print("Linear Regression:")
print(f"  Mean Squared Error: {mse_lr}")
print(f"  R-squared: {r2_lr}\n")


Linear Regression:
  Mean Squared Error: 265.56582435912117
  R-squared: 0.8938071302463938



In [65]:
from sklearn.tree import DecisionTreeRegressor

dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(x_train, y_train)

y_pred_dt = dt_model.predict(x_test)

mse_dt = mean_squared_error(y_test, y_pred_dt)
r2_dt = r2_score(y_test, y_pred_dt)

print("Decision Tree Regressor:")
print(f"  Mean Squared Error: {mse_dt}")
print(f"  R-squared: {r2_dt}\n")


Decision Tree Regressor:
  Mean Squared Error: 0.125417439703154
  R-squared: 0.9999498488261004



In [67]:
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(random_state=42)
gb_model.fit(x_train, y_train)

y_pred_gb = gb_model.predict(x_test)

mse_gb = mean_squared_error(y_test, y_pred_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print("Gradient Boosting Regressor:")
print(f"  Mean Squared Error: {mse_gb}")
print(f"  R-squared: {r2_gb}\n")


Gradient Boosting Regressor:
  Mean Squared Error: 3.8309916843987475
  R-squared: 0.998468086012384



In [69]:
from xgboost import XGBRegressor


xgb_model = XGBRegressor(random_state=42)
xgb_model.fit(x_train, y_train)

y_pred_xgb = xgb_model.predict(x_test)

mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print("XGBoost Regressor:")
print(f"  Mean Squared Error: {mse_xgb}")
print(f"  R-squared: {r2_xgb}\n")


XGBoost Regressor:
  Mean Squared Error: 1.504919227895351
  R-squared: 0.999398221921276

